In [1]:
import tensorflow as tf 
from tensorflow import keras
import pandas as pd 
import numpy as np 
import os
import matplotlib.pyplot as plt 
import time

C:\Users\DELL\AppData\Roaming\Python\Python313\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\DELL\AppData\Roaming\Python\Python313\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\DELL\AppData\Roaming\Python\Python313\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please upd

In [2]:
df = pd.read_csv(r"C:/Users/DELL/OneDrive/Desktop/train.csv")

In [3]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
df.shape

(7613, 5)

In [5]:
print((df.target == 1).sum()) # disaster
print((df.target == 0).sum()) # no disaster

3271
4342


In [7]:
import re  # Regular Expression
import string

def remove_url(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)

def remove_punc(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
pattern = re.compile(r"https?://(\S+|www)\.\S+")
for t in df.text:
    matches = pattern.findall(t)
    for match in matches:
        print("1")
        print(t)
        print("2")
        print(match)
        print("3")
        print(pattern.sub(r"", t))
    if len(matches) > 0:
        break

1
@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C
2
t
3
@bbcmtd Wholesale Markets ablaze 


In [9]:
df['text'] = df.text.map(remove_url)        # map(lambda x: remove_url(x))
df['text'] = df.text.map(remove_punc)
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this earthquake Ma...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask Canada,1
2,5,NaN,NaN,All residents asked to shelter in place are be...,1
3,6,NaN,NaN,13000 people receive wildfires evacuation orde...,1
4,7,NaN,NaN,Just got sent this photo from Ruby Alaska as s...,1


In [10]:
# remove stopwords 
# (Stopwords are the English words which does not add much meaning to a sentence. They can safely be ignored           without sacrificing the meaning of the sentence. For example, the words like the, he, have etc. Such words are      already captured this in corpus named corpus.
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))

def remove_stopwords(text):
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [11]:
df["text"] = df.text.map(remove_stopwords)
df.text

0            deeds reason earthquake may allah forgive us
1                   forest fire near la ronge sask canada
2       residents asked shelter place notified officer...
3       13000 people receive wildfires evacuation orde...
4       got sent photo ruby alaska smoke wildfires pou...
                              ...                        
7608    two giant cranes holding bridge collapse nearb...
7609    ariaahrary thetawniest control wild fires cali...
7610                      m194 0104 utc5km volcano hawaii
7611    police investigating ebike collided car little...
7612    latest homes razed northern california wildfir...
Name: text, Length: 7613, dtype: object

In [12]:
from collections import Counter

# Count unique words
def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count

counter = counter_word(df.text)

In [13]:
len(counter)

17971

In [14]:
# counter

In [15]:
counter.most_common(5)

[('like', 345), ('im', 299), ('amp', 298), ('fire', 250), ('get', 229)]

In [16]:
num_unique_words = len(counter)

In [17]:
train_size = int(df.shape[0] * 0.8)
train_df = df[:train_size]
val_df = df[train_size:]

train_sentence = train_df.text.to_numpy()
train_labels = train_df.target.to_numpy()
val_sentence = val_df.text.to_numpy()
val_labels = val_df.target.to_numpy()

In [18]:
train_sentence.shape, val_sentence.shape
train_sentence

array(['deeds reason earthquake may allah forgive us',
       'forest fire near la ronge sask canada',
       'residents asked shelter place notified officers evacuation shelter place orders expected',
       ..., 'feel like sinking unhappiness take quiz',
       'sinking music video tv career brooke hogan thanking dad free publicityalthough doubt help',
       'supernovalester feel bad literally feel feeling heart sinking bc didnt get anyone ugh jfc'],
      shape=(6090,), dtype=object)

In [19]:
# tokenize 

from tensorflow.keras.preprocessing.text import Tokenizer

# vectorize a text corpus by turning each text into a sequence of interger
tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(train_sentence)  # fit only to training

In [20]:
# each word has unique index
word_index = tokenizer.word_index

In [21]:
# word_index

In [22]:
train_sequence = tokenizer.texts_to_sequences(train_sentence)
val_sequence = tokenizer.texts_to_sequences(val_sentence)

In [23]:
print(train_sentence[0:5])
print(train_sequence[0:5])

['deeds reason earthquake may allah forgive us'
 'forest fire near la ronge sask canada'
 'residents asked shelter place notified officers evacuation shelter place orders expected'
 '13000 people receive wildfires evacuation orders california'
 'got sent photo ruby alaska smoke wildfires pours school']
[[3739, 696, 235, 41, 1282, 3740, 14], [71, 3, 129, 576, 5670, 5671, 1283], [1448, 1186, 1882, 495, 5672, 1449, 116, 1882, 495, 976, 1187], [2243, 8, 3741, 1070, 116, 976, 24], [27, 1071, 358, 5673, 1635, 892, 1070, 5674, 91]]


In [24]:
# pad the sentence to have the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Maximum number of words in a suquence
max_length = 20

train_padded = pad_sequences(train_sequence, maxlen=max_length, padding="post", truncating="post")
val_padded = pad_sequences(val_sequence, maxlen=max_length, padding="post", truncating="post")
train_padded.shape, val_padded.shape

((6090, 20), (1523, 20))

In [25]:
train_padded[0]

array([3739,  696,  235,   41, 1282, 3740,   14,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [26]:
print(train_sentence[10])
print(train_sequence[10])
print(train_padded[10])

three people died heat wave far
[520, 8, 395, 156, 297, 411]
[520   8 395 156 297 411   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]


In [27]:
# Check reversing the indices

# flip key, values
reverse_word_index = dict([(idx, word) for word, idx in word_index.items()])

In [28]:
# reverse_word_index

In [29]:
def decode(sequence):
    return ' '.join([reverse_word_index.get(idx, "?") for idx in sequence])

In [30]:
decode_text = decode(train_sequence[10])
print(train_sequence[10])
print(decode_text)

[520, 8, 395, 156, 297, 411]
three people died heat wave far


In [31]:
# Create LSTM Model
from tensorflow.keras import layers

# Embedding: https://www.tensorflow.org/tutorials/text/word_embeddings
# Turns positive integers (indexes) into dense vectors of fixed size. (other approach could be one-hot-encoding)

model = keras.models.Sequential()
model.add(layers.Embedding(num_unique_words, 32, input_length=max_length))

# The layer will take as input an integer matrix of size (batch, input_length)
# and  the largest integer (i.e. word index) in the input should be no longer than num_words (vocabulary size)
# Now model.output_shape is (None, input_length, 32), where None is tha batch dimension

model.add(layers.LSTM(64, dropout=0.1))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

C:\Users\DELL\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [33]:
loss = keras.losses.BinaryCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(learning_rate=0.001)
metrics = ['accuracy']

model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [34]:
model.fit(train_padded, train_labels, epochs=20, validation_data=(val_padded, val_labels), verbose=2)

Epoch 1/20
191/191 - 15s - 80ms/step - accuracy: 0.7113 - loss: 0.5562 - val_accuracy: 0.7827 - val_loss: 0.4633
Epoch 2/20
191/191 - 6s - 33ms/step - accuracy: 0.8847 - loss: 0.2965 - val_accuracy: 0.7859 - val_loss: 0.5568
Epoch 3/20
191/191 - 6s - 31ms/step - accuracy: 0.9450 - loss: 0.1675 - val_accuracy: 0.6940 - val_loss: 0.8554
Epoch 4/20
191/191 - 6s - 34ms/step - accuracy: 0.9626 - loss: 0.1207 - val_accuracy: 0.7689 - val_loss: 0.6581
Epoch 5/20
191/191 - 10s - 52ms/step - accuracy: 0.9718 - loss: 0.0919 - val_accuracy: 0.7459 - val_loss: 0.8196
Epoch 6/20
191/191 - 6s - 33ms/step - accuracy: 0.9762 - loss: 0.0810 - val_accuracy: 0.7275 - val_loss: 0.9437
Epoch 7/20
191/191 - 6s - 31ms/step - accuracy: 0.9755 - loss: 0.0760 - val_accuracy: 0.7584 - val_loss: 0.6582
Epoch 8/20
191/191 - 7s - 35ms/step - accuracy: 0.9780 - loss: 0.0656 - val_accuracy: 0.7505 - val_loss: 0.6305
Epoch 9/20
191/191 - 11s - 55ms/step - accuracy: 0.9795 - loss: 0.0591 - val_accuracy: 0.7511 - val_lo

In [35]:
predictions = model.predict(train_padded)
predictions = [1 if p> 0.5 else 0 for p in predictions]

191/191 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step


In [36]:
print(train_sentence[10:20])

['three people died heat wave far'
 'haha south tampa getting flooded hah wait second live south tampa gonna gonna fvck flooding'
 'raining flooding florida tampabay tampa 18 19 days ive lost count'
 'flood bago myanmar arrived bago'
 'damage school bus 80 multi car crash breaking' 'whats man' 'love fruits'
 'summer lovely' 'car fast' 'goooooooaaaaaal']


In [37]:
print(train_labels[10:20])
print(predictions[10:20])

[1 1 1 1 1 0 0 0 0 0]
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
